In [1]:
import os
from openmm.app import *
from openmm import *
from openmm.unit import *
from pdbfixer import PDBFixer

# Preparing Barnase 

## Obtaining best chain

From barnase-barstar pdb previously analyzed, it was identifed that the "best" barnase chain contain 110 residues and 878 atoms, so the task is to identify it from the 12 chains in the pdb. 

In [2]:
fixer = PDBFixer(filename='1brs.pdb')

In [3]:
fixer.topology

<Topology; 12 chains, 1101 residues, 5149 atoms, 4738 bonds>

In [4]:
fixer.removeChains([0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

In [5]:
fixer.topology

<Topology; 1 chains, 110 residues, 878 atoms, 899 bonds>

Although manually, the barnase chain was identified.

<div class="alert alert-info">
<strong>Question:</strong> Is there a more efficient way to identify the chain were looking for?
</div>

<div class="alert alert-info">
<strong>Question:</strong> How call a list in an object in order to print all the residues?
</div>

## Editing

Using PDBFixer, we can add residues, atoms and water molecules to our system.

In [6]:
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)

In [7]:
fixer.topology

<Topology; 1 chains, 110 residues, 1727 atoms, 1748 bonds>

Comparing the number of atoms in the topology, we can be sure that our system has been complemented. If we change the pH there will be more atoms due the extra hydrogens.

In [8]:
PDBFile.writeFile(fixer.topology, fixer.positions, open('barnase.pdb', 'w'))

## Finishing with Modeller

In order to build a system with an specific water model and box shape, Modeller is a great tool.

In [9]:
pdb = PDBFile('barnase.pdb')

In [10]:
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')

In [11]:
modeller = Modeller(pdb.topology, pdb.positions)

In [12]:
modeller.addSolvent(forcefield, model='tip3p')

<div class="alert alert-warning">
<strong>Task:</strong> Can't determine system's charge.
</div>

In [13]:
charge = modeller.getTopology()

In [14]:
modeller.topology

<Topology; 3 chains, 23242 residues, 71119 atoms, 48008 bonds>